In [1]:
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from mtcnn.mtcnn import MTCNN
import numpy as np
import tensorflow as tf
from numpy import asarray
from PIL import Image
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
from scipy.spatial.distance import cosine
import cv2

In [2]:
tf.compat.v1.disable_eager_execution() #optimization purposes

In [3]:
def highlight_faces(image, faces):
  
    for face in faces:
        x, y, width, height = face['box']
        cv2.rectangle(image,(x,y),(x + width, y + height),color=(0,0,255),thickness=4)
  

In [4]:
def change_res(cap, width, height):
    cap.set(3, width)
    cap.set(4, height)

In [6]:
def extract_face_from_image(image, face_infos, required_size=(224, 224)):
  # load image and detect faces
  
    face_images = []

    for face in face_infos:
        # extract the bounding box from the requested face
        x1, y1, width, height = face['box']
        x2, y2 = x1 + width, y1 + height

        # extract the face
        face_boundary = image[y1:y2, x1:x2]

        # resize pixels to the model size
        face_image = Image.fromarray(face_boundary)
        face_image = face_image.resize(required_size)
        face_array = asarray(face_image)
        face_images.append(face_array)

    return face_images

In [7]:
model = VGGFace(model='resnet50',
      include_top=False,
      input_shape=(224, 224, 3),
      pooling='avg')

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
def get_face_mapping(faces, target_face):
    faces = np.append(faces, target_face, axis=0)
    samples = asarray(faces, 'float32')

  # prepare the data for the model
    samples = preprocess_input(samples, version=2)

  # perform prediction
    return model.predict(samples)

In [9]:
threshold = 0.5

In [12]:
cap = cv2.VideoCapture(0)

change_res(cap, 200, 130)
detector = MTCNN()

target = cv2.imread("images/blade_runner2049_2.jpg") # read students face
target = cv2.cvtColor(target, cv2.COLOR_BGR2RGB)
target_face_info = detector.detect_faces(target) # get face boundaries

target_face = np.array(extract_face_from_image(target, target_face_info)) # extract face and append to array

face_matched = False

while True:
    
    ret, frame = cap.read() # read from camera
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    #detect faces
    face_infos = detector.detect_faces(frame_rgb) 
    face_infos = np.array(face_infos)
    
    if len(face_infos) > 0 :
        
        face_images = extract_face_from_image(frame_rgb, face_infos)
        # TODO: flush faces except target in every loop                   
        #faces = np.append(face_infos, target_face, axis=0)
        
        face_vectors = get_face_mapping(face_images, target_face)
         
        face_count = face_vectors.shape[0]
                                
        for i in range(0, face_count):
            for j in range(i, face_count):
                if i == j:
                    continue
                if cosine(face_vectors[i], face_vectors[j]) <= threshold:
                    print("Face " + str(i) + " and Face " + str(j) + " are Matched")
                    face_matched = True
                
        if not face_matched:
            print("no face matched")
                                
    #highlight_faces(frame, faces)
    
    for face in face_infos:
        x, y, width, height = face['box']
        cv2.rectangle(frame, (x,y), (x + width, y + height), color=(0,0,255), thickness=4)
        
    cv2.imshow("frame", frame)
    # press ESC to close window
    if cv2.waitKey(1) & 0xFF == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face matched
no face 